In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data preprocessing, read csv
import pandas as pd
import numpy as np

customer = pd.read_csv ('/content/drive/MyDrive/Datathon/Dataset/olist_customers_dataset.csv')
item = pd.read_csv ('/content/drive/MyDrive/Datathon/Dataset/olist_order_items_dataset.csv')
payment = pd.read_csv ('/content/drive/MyDrive/Datathon/Dataset/olist_order_payments_dataset.csv')
review = pd.read_csv ('/content/drive/MyDrive/Datathon/Dataset/olist_order_reviews_dataset.csv')#drop low score
orders = pd.read_csv ('/content/drive/MyDrive/Datathon/Dataset/olist_orders_dataset.csv')
products = pd.read_csv ('/content/drive/MyDrive/Datathon/Dataset/olist_products_dataset.csv')
translation = pd.read_csv ('/content/drive/MyDrive/Datathon/Dataset/product_category_name_translation.csv')

In [ ]:
customer = customer.iloc[:,[0,1]]

In [ ]:
orders = orders.iloc[:,[0,1,]]

In [ ]:
customer = pd.merge(customer, orders, on=["customer_id"], how = "inner")

In [ ]:
customer = pd.merge(customer, item, on=["order_id"], how = "inner").dropna()
customer = customer.iloc[:, [1,2,4,7]]

In [ ]:
customer = pd.merge(customer, products, on=["product_id"], how = "inner")

In [ ]:
customer = customer.iloc[:,[0,1,2,3,4]]

In [ ]:
purchase_his  = customer.dropna()

In [ ]:
purchase_his2 = purchase_his

In [ ]:
purchase_his = purchase_his.set_index('product_category_name').join(translation.set_index('product_category_name'))

In [ ]:
purchase_his.reset_index(level = 0, inplace = True)

In [ ]:
purchase_his = purchase_his.dropna()
purchase_his = purchase_his.drop_duplicates(subset = ["product_id"])

In [ ]:
products_sub = purchase_his.iloc[:,[3,4,5]]
temp_num = products_sub.groupby(['product_category_name_english'])['product_id'].count().reset_index(level = 0)
temp_num=temp_num.sort_values(by=['product_id'])
Over_3=temp_num[temp_num["product_id"] > 3]

In [ ]:
products_sub["cluster"] = 2
productsX = []
for i in Over_3["product_category_name_english"]:
  productsX.append(products_sub[products_sub["product_category_name_english"]== i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#Use K-Means clustering
#Classify all the product categories that have at least three products based on price range: cheap, medium, expensive (0, 1, 2) 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
for i in productsX:
  kmeans = KMeans(n_clusters=3, random_state=777).fit(i.iloc[:,[1]])
  i["cluster"]=kmeans.labels_

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [ ]:
dvd = products_sub[products_sub["product_category_name_english"] == "cds_dvds_musicals"]
sercurity = products_sub[products_sub["product_category_name_english"] == "security_and_services"]
productsX.append(dvd)
productsX.append(sercurity)
products_sub_clustered = pd.concat(productsX)

In [ ]:
products_sub_clustered = products_sub_clustered.iloc[:,[0,2,3]]

In [ ]:
products_sub_clustered

,product_id,product_category_name_english,cluster
34808,ec5b3c8bb77ad22278f6e9d719a83de3,home_comfort_2,1
34834,15dc5bc6f0f1e1a0d0255098726060c7,home_comfort_2,1
34835,4fb3bad6b502eaca3b6d7d87bc1613a4,home_comfort_2,1
34836,2072d4792ab7893ddbfc178948e0eb86,home_comfort_2,0
34837,19de58ea609a59ce31f93672af50c09a,home_comfort_2,2
...,...,...,...
34372,25c9bd06b9af867e935cca386b6acece,bed_bath_table,2
34373,ea43b87d74397400228d42e26adb7f24,bed_bath_table,0
35442,1dceebcc5f23c02ea23e16d5bedca000,cds_dvds_musicals,2
98990,6c7a0a349ad11817745e3ad58abd5c79,security_and_services,2


In [ ]:
product_names = products_sub_clustered["product_category_name_english"].value_counts().index

In [ ]:
product_names

Index(['bed_bath_table', 'sports_leisure', 'furniture_decor', 'health_beauty',
       'housewares', 'auto', 'computers_accessories', 'toys', 'watches_gifts',
       'telephony', 'baby', 'perfumery', 'stationery',
       'fashion_bags_accessories', 'cool_stuff', 'garden_tools', 'pet_shop',
       'electronics', 'construction_tools_construction', 'home_appliances',
       'luggage_accessories', 'consoles_games', 'office_furniture',
       'musical_instruments', 'small_appliances', 'home_construction',
       'books_general_interest', 'fashion_shoes', 'furniture_living_room',
       'air_conditioning', 'books_technical', 'fixed_telephony',
       'home_confort', 'food_drink', 'market_place', 'fashion_male_clothing',
       'kitchen_dining_laundry_garden_furniture', 'signaling_and_security',
       'construction_tools_safety', 'home_appliances_2',
       'costruction_tools_garden', 'food', 'drinks',
       'construction_tools_lights', 'agro_industry_and_commerce',
       'industry_commerce

In [ ]:
product_names
product_names_new = []
for i in product_names:
  product_names_new.append(i.replace("_", " ")) 
product_names_new[52] = "construction tools tools"
product_names_new[40] = "construction tools garden"
product_names_new[57] = "fashion female clothing"

In [ ]:
#Natural Language Processing: calculate the similarity between the meaning of each product category
from scipy import spatial
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-50") #choose from multiple models https://github.com/RaRe-Technologies/gensim-data

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
def preprocess(s):
    return [i.lower() for i in s.split()]

def get_vector(s):
    return np.sum(np.array([model[i] for i in preprocess(s)]), axis=0)
  
all_result = np.zeros(shape = (71,71))
for i in range(len(product_names_new)):
  for j in range(len(product_names_new)):
     all_result[i][j]= 1 - spatial.distance.cosine(get_vector(product_names_new[i]), get_vector(product_names_new[j]))

In [ ]:
all_result
#The result is a 71x71(since we have 71 categories) distance matrix indicating the distance between the current category and each
#of other categories: the larger the number, the closer these two words are in the vector space, thus these two words are more similar

array([[1.        , 0.47006533, 0.51970279, ..., 0.50716257, 0.49204898,
        0.22214805],
       [0.47006533, 1.        , 0.42720658, ..., 0.54747117, 0.56685537,
        0.30704537],
       [0.51970279, 0.42720658, 1.        , ..., 0.68819845, 0.30474111,
        0.41380927],
       ...,
       [0.50716257, 0.54747117, 0.68819845, ..., 1.        , 0.4743664 ,
        0.47316551],
       [0.49204898, 0.56685537, 0.30474111, ..., 0.4743664 , 1.        ,
        0.23354973],
       [0.22214805, 0.30704537, 0.41380927, ..., 0.47316551, 0.23354973,
        1.        ]])

In [ ]:
purchasetimes = item.groupby(['product_id']).count().reset_index(level = 0)
temp2 = purchasetimes.dropna().iloc[:,0:2]
review_score = review.iloc[:,1:3]
order_product = item.iloc[:,0:3]

result = pd.merge(review_score, order_product, on="order_id")
result = result.dropna()
result = result.iloc[:, [1,3]]
result = result.groupby(['product_id']).mean().reset_index(level = 0)
finalresult = pd.merge(result, temp2, on="product_id")
finalresult = finalresult.rename(columns={"order_id":"counts"})
products_sub_clustered = pd.merge(finalresult, products_sub_clustered, on=["product_id"], how = "inner")

In [ ]:
#Define the popularity score function: popularity = log( # of purchase * 1/(6 -median rating)) )* subcat_weight (subcat_weight will be multiplied later, not in this current cell)
products_sub_clustered["popularity_score"] = np.log(products_sub_clustered["counts"] + 1) * (1/(6-products_sub_clustered["review_score"]))
products_sub_clustered = products_sub_clustered.iloc[:,[0,3,4,5]]

In [ ]:
products_sub_clustered
#This is the product table after we processed its belonged cluster and calculated its unique popularity score

,product_id,product_category_name_english,cluster,popularity_score
0,00066f42aeeb9f3007548bb9d3f33c38,perfumery,0,0.693147
1,00088930e925c41fd95ebfe695fd2655,auto,0,0.346574
2,0009406fd7479715e4bef61dd91f2462,bed_bath_table,2,0.138629
3,000b8f95fcb9e0096488278317764d19,housewares,1,1.098612
4,000d9be29b5207b54e86aa1b1ac54872,watches_gifts,1,0.693147
...,...,...,...,...
32166,fff6177642830a9a94a0f2cba5e476d1,cool_stuff,0,0.732408
32167,fff81cc3158d2725c0655ab9ba0f712c,health_beauty,0,0.346574
32168,fff9553ac224cec9d15d49f5a263411f,fashion_bags_accessories,0,0.693147
32169,fffdb2d0ec8d6a61f0a0a0db3f25b441,computers_accessories,1,1.791759


In [ ]:
# Revise some misspelled words in the category name translation to better process the category names in NLP task
products_sub_clustered["product_category_name_english"] = products_sub_clustered["product_category_name_english"].apply(lambda x: x.replace("fasion", "fashion"))

In [ ]:
products_sub_clustered["product_category_name_english"] = products_sub_clustered["product_category_name_english"].apply(lambda x: x.replace("costruction", "construction"))

In [ ]:
products_sub_clustered["product_category_name_english"] = products_sub_clustered["product_category_name_english"].apply(lambda x: x.replace("_", " "))

In [ ]:
products_sub_clustered

,product_id,product_category_name_english,cluster,popularity_score
0,00066f42aeeb9f3007548bb9d3f33c38,perfumery,0,0.693147
1,00088930e925c41fd95ebfe695fd2655,auto,0,0.346574
2,0009406fd7479715e4bef61dd91f2462,bed bath table,2,0.138629
3,000b8f95fcb9e0096488278317764d19,housewares,1,1.098612
4,000d9be29b5207b54e86aa1b1ac54872,watches gifts,1,0.693147
...,...,...,...,...
32166,fff6177642830a9a94a0f2cba5e476d1,cool stuff,0,0.732408
32167,fff81cc3158d2725c0655ab9ba0f712c,health beauty,0,0.346574
32168,fff9553ac224cec9d15d49f5a263411f,fashion bags accessories,0,0.693147
32169,fffdb2d0ec8d6a61f0a0a0db3f25b441,computers accessories,1,1.791759


In [ ]:
purchase_his2_customer = purchase_his2.iloc[:,[0,2]]

In [ ]:
freq = purchase_his2_customer["customer_unique_id"].value_counts()
onetime_buyer = freq[freq == 1].index
moretime_buyer = freq[freq > 1].index

In [ ]:
customer1 = pd.DataFrame()
customer2 = pd.DataFrame()
customer1 = purchase_his2_customer[purchase_his2_customer.customer_unique_id.isin(onetime_buyer)]
customer2 = purchase_his2_customer[purchase_his2_customer.customer_unique_id.isin(moretime_buyer)]
customer2_drop = customer2.drop_duplicates(subset = ["customer_unique_id"])
customer2_joint = pd.merge(customer2, customer2, on=["customer_unique_id"], how = "inner")
customer2_joint = customer2_joint[customer2_joint["product_id_x"] != customer2_joint["product_id_y"]]
customer2_joint = customer2_joint.drop_duplicates(subset = ["customer_unique_id"])

In [ ]:
#Get all the multiple-time buyers and the purchased items
customer2

,customer_unique_id,product_id
1,9eae34bbd3a474ec5d07949ca7de67c0,a9516a079e37a9c9c36b9b78b10169e8
2,9eae34bbd3a474ec5d07949ca7de67c0,a9516a079e37a9c9c36b9b78b10169e8
9,31318a0597cd9d50ce4cfd03c80fe780,a9516a079e37a9c9c36b9b78b10169e8
10,31318a0597cd9d50ce4cfd03c80fe780,a9516a079e37a9c9c36b9b78b10169e8
11,83aca63b2193ea126b287f3ae61528d9,a9516a079e37a9c9c36b9b78b10169e8
...,...,...
112630,0ee57f62666561b72f2ceacad0230cbf,585e7f7e97e55cb9e74556236b43b0be
112638,40eeaec8ded23f98865c1ebdb041f6ea,d88df6347b8ee090e7a79d4493e1abe2
112639,40eeaec8ded23f98865c1ebdb041f6ea,d88df6347b8ee090e7a79d4493e1abe2
112641,a5ba329297100ea689fa263768b35b8b,90ed53d34bfcb4fa1e90656068dd04bc


In [ ]:
#See all the products that this customer has bought
customer2[customer2["customer_unique_id"]=="c8460e4251689ba205045f3ea17884a1"]

,customer_unique_id,product_id
9442,c8460e4251689ba205045f3ea17884a1,e7cc48a9daff5436f63d3aad9426f28b
9443,c8460e4251689ba205045f3ea17884a1,e7cc48a9daff5436f63d3aad9426f28b
9444,c8460e4251689ba205045f3ea17884a1,e7cc48a9daff5436f63d3aad9426f28b
9445,c8460e4251689ba205045f3ea17884a1,e7cc48a9daff5436f63d3aad9426f28b
9446,c8460e4251689ba205045f3ea17884a1,e7cc48a9daff5436f63d3aad9426f28b
9447,c8460e4251689ba205045f3ea17884a1,e7cc48a9daff5436f63d3aad9426f28b
9472,c8460e4251689ba205045f3ea17884a1,e7cc48a9daff5436f63d3aad9426f28b
9473,c8460e4251689ba205045f3ea17884a1,e7cc48a9daff5436f63d3aad9426f28b
9474,c8460e4251689ba205045f3ea17884a1,e7cc48a9daff5436f63d3aad9426f28b
9475,c8460e4251689ba205045f3ea17884a1,e7cc48a9daff5436f63d3aad9426f28b


In [ ]:
#Get all one-time buyers and the purchased item
customer1

,customer_unique_id,product_id
0,861eff4711a542e4b93843c6dd7febb0,a9516a079e37a9c9c36b9b78b10169e8
3,3c799d181c34d51f6d44bbbc563024db,a9516a079e37a9c9c36b9b78b10169e8
4,23397e992b09769faf5e66f9e171a241,a9516a079e37a9c9c36b9b78b10169e8
5,567ab47ca4deb92d46dbf54dce07d0a7,a9516a079e37a9c9c36b9b78b10169e8
6,f40ab89b622248b7ca125af4b486b887,a9516a079e37a9c9c36b9b78b10169e8
...,...,...
112645,fb354969e06f2093c0083cbfbb91864e,9682ad2500ae8b2609e6a88eb0cbc5bb
112646,5f2971f9805e3ccb030226e30c8e8390,aea06073397f809424f946979354c9f0
112647,c64ede6d0ae8901b1b6fb03528c1b7e6,bbf975bffd2ae9ee52f513ae5c8a4b27
112648,046470763123d3d6364f89095b4e47ab,f6e0a9ce8a6e91c3a0ca2d3005911d20


In [ ]:
#Randomly select two bought products from customers who has 2 or more purchase histories
select_2 = customer2.groupby("customer_unique_id").sample(n=2)
select_2

,customer_unique_id,product_id
77313,00053a61a98854899e70ed204dd4bafe,62984ea1bba7fcea1f5b57084d3bf885
12177,00053a61a98854899e70ed204dd4bafe,58727e154e8e85d84052cd22b0136c84
2460,000de6019bb59f34c099a907c151d855,af0a917aec9cea3b353ece61a8825326
64258,000de6019bb59f34c099a907c151d855,9e572ff4654f7064419d97a891a8b0fc
32995,000fbf0473c10fc1ab6f8d2d286ce20c,38021cef829efa264df6f9d21c4df6e9
...,...,...
97369,fff5eb4918b2bf4b2da476788d42051c,bc3c6d2a621414f2e1df7a8a32a2828e
54799,fff7219c86179ca6441b8f37823ba3d3,056d012d264624accb7f73d31caee034
10254,fff7219c86179ca6441b8f37823ba3d3,592962829d5a715304344e656e39108a
108423,fffcf5a5ff07b0908bd4e2dbc735a684,bbc5419bc92d76e27795c9ec2c233c64


In [ ]:
customer1_rec = pd.DataFrame()
tests = customer1.iloc[0:100]

In [ ]:
products_sub_clustered[products_sub_clustered["product_id"] == customer1.iloc[5297]["product_id"]]

,product_id,product_category_name_english,cluster,popularity_score
14473,726e355b119d1f8186283de59fc5b702,luggage accessories,1,1.098612


In [ ]:
one_time_rec = pd.DataFrame()
for i in range(len(customer1)):
  #Get the one-time purchase history from each customer
  bought = products_sub_clustered[products_sub_clustered["product_id"] == customer1.iloc[i]["product_id"]]
  try:
    #Get the category of the bought product, push recommendations based on this same category
    cate = bought["product_category_name_english"].iloc[0]
    #Get the subcategory labels we clustered before (cheap, medium, expensive), recommend products with the same label
    subcate = bought["cluster"].iloc[0]
    #Get all products that can satisfy 1).within the same category of previous purchase 2).within the same price range(label)
    recommend = products_sub_clustered[(products_sub_clustered["product_category_name_english"] == cate) 
                                      & (products_sub_clustered["cluster"] == subcate)]
    #Get 20 products with highest popularity score we calculated before                          
    recommend_top20 = recommend.nlargest(n=20, columns=['popularity_score']).iloc[:,0]
    one_time_rec.reset_index(drop=True,inplace = True)
    recommend_top20.reset_index(drop=True, inplace = True)
    #Concatenate into dataframe table
    one_time_rec = pd.concat([one_time_rec, recommend_top20.to_frame().T], axis = 0, ignore_index=True)
  except:
    one_time_rec = pd.concat([one_time_rec, recommend_top20.to_frame().T], axis = 0, ignore_index=True)
 

In [ ]:
one_time_rec
#This block will generate 20 recommended products for 82433 customers who has one purchase history on the retailer platform.

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,475e8a9ddbebf13af503d1c7eccadb1a,6f842ad54a86bf94d9254744d910ec56,aba86c093ccdbac75b09111d57e50004,72ef87b1953a2844ed089bd6dcf73913,b03f486273ee4263286c00383ef34f37,f2399cabe080029634d12cec4f60dbf0,b92a7304ebad1ca5b393b53b2de5c70a,8ed094bfe076c568f6bb10feada3f75d,7c898e0b8ea203dd94ba846627fc34d1,cbdb9bf611328898bd4b32a384506b3a,a9516a079e37a9c9c36b9b78b10169e8,1bfb290d7273a442c874dbe74b4abae6,2b7aa376a6e728560bddb5558cc48e89,0335b2f76fdeed85180abe8c6c444aa4,2a5806f10d0f00e5ad032dd2e3c8806e,1bcb50aa4fa11ec2f767d6eea810d8b7,7dda94d5b1861c308661fedb3944df60,3eef0cb94ba82de806bb30ab743c7655,b114bf337c0626166abe574eee9e3f32,02ce0c74e59d6cf6bc9026601414a9c4
1,475e8a9ddbebf13af503d1c7eccadb1a,6f842ad54a86bf94d9254744d910ec56,aba86c093ccdbac75b09111d57e50004,72ef87b1953a2844ed089bd6dcf73913,b03f486273ee4263286c00383ef34f37,f2399cabe080029634d12cec4f60dbf0,b92a7304ebad1ca5b393b53b2de5c70a,8ed094bfe076c568f6bb10feada3f75d,7c898e0b8ea203dd94ba846627fc34d1,cbdb9bf611328898bd4b32a384506b3a,a9516a079e37a9c9c36b9b78b10169e8,1bfb290d7273a442c874dbe74b4abae6,2b7aa376a6e728560bddb5558cc48e89,0335b2f76fdeed85180abe8c6c444aa4,2a5806f10d0f00e5ad032dd2e3c8806e,1bcb50aa4fa11ec2f767d6eea810d8b7,7dda94d5b1861c308661fedb3944df60,3eef0cb94ba82de806bb30ab743c7655,b114bf337c0626166abe574eee9e3f32,02ce0c74e59d6cf6bc9026601414a9c4
2,475e8a9ddbebf13af503d1c7eccadb1a,6f842ad54a86bf94d9254744d910ec56,aba86c093ccdbac75b09111d57e50004,72ef87b1953a2844ed089bd6dcf73913,b03f486273ee4263286c00383ef34f37,f2399cabe080029634d12cec4f60dbf0,b92a7304ebad1ca5b393b53b2de5c70a,8ed094bfe076c568f6bb10feada3f75d,7c898e0b8ea203dd94ba846627fc34d1,cbdb9bf611328898bd4b32a384506b3a,a9516a079e37a9c9c36b9b78b10169e8,1bfb290d7273a442c874dbe74b4abae6,2b7aa376a6e728560bddb5558cc48e89,0335b2f76fdeed85180abe8c6c444aa4,2a5806f10d0f00e5ad032dd2e3c8806e,1bcb50aa4fa11ec2f767d6eea810d8b7,7dda94d5b1861c308661fedb3944df60,3eef0cb94ba82de806bb30ab743c7655,b114bf337c0626166abe574eee9e3f32,02ce0c74e59d6cf6bc9026601414a9c4
3,475e8a9ddbebf13af503d1c7eccadb1a,6f842ad54a86bf94d9254744d910ec56,aba86c093ccdbac75b09111d57e50004,72ef87b1953a2844ed089bd6dcf73913,b03f486273ee4263286c00383ef34f37,f2399cabe080029634d12cec4f60dbf0,b92a7304ebad1ca5b393b53b2de5c70a,8ed094bfe076c568f6bb10feada3f75d,7c898e0b8ea203dd94ba846627fc34d1,cbdb9bf611328898bd4b32a384506b3a,a9516a079e37a9c9c36b9b78b10169e8,1bfb290d7273a442c874dbe74b4abae6,2b7aa376a6e728560bddb5558cc48e89,0335b2f76fdeed85180abe8c6c444aa4,2a5806f10d0f00e5ad032dd2e3c8806e,1bcb50aa4fa11ec2f767d6eea810d8b7,7dda94d5b1861c308661fedb3944df60,3eef0cb94ba82de806bb30ab743c7655,b114bf337c0626166abe574eee9e3f32,02ce0c74e59d6cf6bc9026601414a9c4
4,475e8a9ddbebf13af503d1c7eccadb1a,6f842ad54a86bf94d9254744d910ec56,aba86c093ccdbac75b09111d57e50004,72ef87b1953a2844ed089bd6dcf73913,b03f486273ee4263286c00383ef34f37,f2399cabe080029634d12cec4f60dbf0,b92a7304ebad1ca5b393b53b2de5c70a,8ed094bfe076c568f6bb10feada3f75d,7c898e0b8ea203dd94ba846627fc34d1,cbdb9bf611328898bd4b32a384506b3a,a9516a079e37a9c9c36b9b78b10169e8,1bfb290d7273a442c874dbe74b4abae6,2b7aa376a6e728560bddb5558cc48e89,0335b2f76fdeed85180abe8c6c444aa4,2a5806f10d0f00e5ad032dd2e3c8806e,1bcb50aa4fa11ec2f767d6eea810d8b7,7dda94d5b1861c308661fedb3944df60,3eef0cb94ba82de806bb30ab743c7655,b114bf337c0626166abe574eee9e3f32,02ce0c74e59d6cf6bc9026601414a9c4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82428,35afc973633aaeb6b877ff57b2793310,873417a13086036760bd9528d4d13d27,9477e72598d2f21dc53f809bd42cf7a2,7248b99ff380f23a1094ff737a1112fb,c6bf8cc852eee5c4ab264e792a9c23a1,828c579b949972bdea5cf2203faf4563,4e191b1e429b88c6eae1a014f823cb7c,d6061ffaf719f75dd074a637c535b644,829b1d3da6a5e54d51055338281a5dfe,ed22e5834e262559c99d7b0975e1032e,f3d786d97bcacbff8312cf48b52a6062,186052b3f166a0f7227e78b2d27652d3,f42892b40225227fe465626ff509a92e,908dd61b840a480fa931ef04389c6266,b3d1d218a4ff07119f546a42c6ee7d7b,9696fc4ef13aa92c9933cc364d18ac40,7c65bcd66be0ad761948b